In [1]:
from tdt_autopsy.logregs_extra_experiments import tidy_results

In [2]:
# Let's load the very anecdotical results from the competition benchmark
#   - Train on the whole training set with the best individual logreg
#   - Test in the tiny competition set (1056 mols, around 100 positives)
#     Anecdotical, but gives us the prize and agrees with more serious cross-val + OOB evaluations.
df = tidy_results()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 24 columns):
data_name                    836 non-null object
model_name                   836 non-null object
binarize_threshold           836 non-null float64
is_counts                    836 non-null bool
allow_unseen_in_folding      836 non-null bool
fold_size                    836 non-null int64
fold_seed                    836 non-null int64
min_radius                   836 non-null float64
max_radius                   836 non-null float64
row_normalizer               836 non-null object
scale                        836 non-null bool
model_sparsity               836 non-null float64
auc                          836 non-null float64
enrichment_1                 836 non-null float64
enrichment_5                 836 non-null float64
enrichment_10                836 non-null float64
bedroc20                     836 non-null float64
rie20                        836 non-null float64
pre_column_

In [3]:
# The table looks like this...
df.sort_values('auc', ascending=False).reset_index(drop=True)

,data_name,model_name,binarize_threshold,is_counts,allow_unseen_in_folding,fold_size,fold_seed,min_radius,max_radius,row_normalizer,...,enrichment_5,enrichment_10,bedroc20,rie20,pre_column_manipulation_s,fit_s,predict_s,eval_s,total_s,id
0,competition-external,tdtl2,0.000000,False,False,32767,3,-inf,inf,none,...,4.931814,4.241360,0.497930,4.437377,2.053404e-02,49.524412,0.029634,0.005873,49.580529,82a4c7e217cdc21947c47ce7ea0b4d6d31b78c46
1,competition-external,tdtl2,0.000000,False,True,32767,3,-inf,4.000000,none,...,4.339996,3.846815,0.456007,4.063771,2.542443e+01,53.052528,0.029086,0.013687,78.519813,bc4d16726ea48f25f1acde1349f281e64f843332
2,competition-external,tdtl2,0.000000,False,False,32767,3,-inf,4.000000,none,...,4.537269,3.846815,0.472054,4.206777,2.502242e+01,48.158864,0.027529,0.011450,73.220338,2d2435692acfaaf65e5dcb1cc5773e9875fe5a98
3,competition-external,tdtl2,0.000000,False,False,8191,2,-inf,4.000000,none,...,4.537269,4.142724,0.459105,4.091384,2.306114e+01,51.061156,0.028166,0.008821,74.159371,7aaf34cd95ddf7afdec484f50ef268bb418b69df
4,competition-external,tdtl2,0.000000,False,True,8191,2,-inf,4.000000,none,...,5.129086,4.142724,0.476087,4.242721,2.454078e+01,49.937829,0.029645,0.008864,74.517201,b6048931251bc27c08367fdaf852ae18a821e1aa
5,competition-external,tdtl2,inf,True,False,4294967295,-1,-inf,inf,none,...,5.129086,4.438633,0.498471,4.442201,4.669809e-02,193.954131,0.001387,0.006913,194.009368,9a6719040ac386cd160e19a2374ed253808f2e38
6,competition-external,tdtl2,0.000000,False,True,32767,2,-inf,inf,none,...,4.537269,4.142724,0.488725,4.355349,9.536743e-07,50.669414,0.026260,0.005205,50.700945,deefc951e3293c257acd8c58fdc1d9cd4186a3c0
7,competition-external,tdtl2,0.000000,False,False,4294967295,-1,-inf,inf,none,...,4.537269,4.241360,0.493071,4.394079,2.929378e-02,190.884758,0.001755,0.007118,190.923148,8e45cb1d2714694b9b8a586f3873e7514c3c6521
8,competition-external,tdtl2,0.000000,False,False,32767,3,-inf,inf,l2,...,4.142724,4.241360,0.485618,4.327660,2.375960e-02,27.374908,0.029118,0.005809,27.433678,2a73d83dc28d7b320b5cf92eb50244dcc0c0680e
9,competition-external,tdtl2,0.000000,False,False,32767,2,-inf,inf,none,...,4.931814,4.044087,0.482462,4.299536,1.959634e-02,48.746283,0.034232,0.007004,48.807203,6fd40e118bfbd1d8abcf9e42b8e5ce95e905f65b


In [4]:
# Lets look at:
# counts vs binary (remember, still not implemented for folding) 
#   => irrelevant for non folding (most values are 1 anyway)
#      result might differ enormously when we allow folding counts
# whether we allow unseen features to add noise in folding
#   => large effect when folding to usual sizes (1024, 2048, 4096)
#      N.B. should we had any small effect of folding seed for no folding, 
#      it would be due to liblinear uncontrollable seed:
#        https://github.com/scikit-learn/scikit-learn/issues/365
#   => the effect is smaller as fold size grows
# how we normalize each row
#   => showed no effect, so to make things more easily graspable, I have moved it to
#      the query from the groupby (look at it back when we add counts to folding)
df.query('model_name == "tdtl2" and '
         'not scale and '
         'max_radius > 1000000 and '
         'row_normalizer == "none"').groupby(['fold_size', 
                                              'is_counts',
                                              'allow_unseen_in_folding']).auc.describe()

count      mean       std  \
fold_size  is_counts allow_unseen_in_folding                              
511        False     False                      4.0  0.692706  0.022528   
                     True                       4.0  0.670714  0.032660   
1023       False     False                      4.0  0.726727  0.018914   
                     True                       4.0  0.711995  0.022059   
2047       False     False                      4.0  0.731092  0.013932   
                     True                       4.0  0.710857  0.017002   
4091       False     False                      4.0  0.770587  0.011615   
                     True                       4.0  0.732038  0.015882   
8191       False     False                      4.0  0.799811  0.018252   
                     True                       4.0  0.777858  0.013391   
16383      False     False                      4.0  0.806804  0.013701   
                     True                       4.0  0.793290  0.007707   
32767      False     False                      4.0  0.823646  0.020626   
                     True                       4.0  0.815546  0.021546   
4294967295 False     False                      1.0  0.834835       NaN   
           True      False                      1.0  0.835415       NaN   

                                                   min       25%       50%  \
fold_size  is_counts allow_unseen_in_folding                                 
511        False     False                    0.661967  0.682852  0.698989   
                     True                     0.629973  0.654061  0.673184   
1023       False     False                    0.714592  0.717049  0.718693   
                     True                     0.695983  0.698619  0.703924   
2047       False     False                    0.710974  0.728570  0.735167   
                     True                     0.685781  0.708268  0.717184   
4091       False     False                    0.757939  0.762122  0.771774   
                     True                     0.714572  0.720660  0.733301   
8191       False     False                    0.776839  0.790104  0.801871   
                     True                     0.759930  0.771453  0.781675   
16383      False     False                    0.796620  0.796822  0.802410   
                     True                     0.785911  0.787746  0.792183   
32767      False     False                    0.794319  0.817280  0.829662   
                     True                     0.786802  0.805541  0.820056   
4294967295 False     False                    0.834835  0.834835  0.834835   
           True      False                    0.835415  0.835415  0.835415   

                                                   75%       max  
fold_size  is_counts allow_unseen_in_folding                      
511        False     False                    0.708844  0.710881  
                     True                     0.689837  0.706516  
1023       False     False                    0.728371  0.754932  
                     True                     0.717301  0.744150  
2047       False     False                    0.737689  0.743062  
                     True                     0.719773  0.723280  
4091       False     False                    0.780239  0.780862  
                     True                     0.744679  0.746980  
8191       False     False                    0.811578  0.818662  
                     True                     0.788080  0.788150  
16383      False     False                    0.812392  0.825774  
                     True                     0.797727  0.802882  
32767      False     False                    0.836027  0.840941  
                     True                     0.830061  0.835270  
4294967295 False     False                    0.834835  0.834835  
           True      False                    0.835415  0.835415

In [5]:
# Or more focused
df.query('model_name == "tdtl2" and '         
         'max_radius > 1000000 and '
         'fold_size == 4091 and '
         'not scale and '
         'row_normalizer == "none"').groupby('allow_unseen_in_folding').auc.mean()


allow_unseen_in_folding
False    0.770587
True     0.732038
Name: auc, dtype: float64

In [6]:
# What about the max radius?
#  - For small fold sizes, larger radii have no to detrimental effect => Due to growing collision rate?
#  - For large fold sizes and no fold, the model is able to figure out how not to over fit too much => clear gains 
df.query('model_name == "tdtl2" and '
         'row_normalizer == "none" and '
         'not scale').groupby(['fold_size', 
                               'is_counts',
                               'allow_unseen_in_folding',
                               'max_radius']).auc.describe()

count      mean  \
fold_size  is_counts allow_unseen_in_folding max_radius                    
511        False     False                   2.000000      4.0  0.690356   
                                             3.000000      4.0  0.693982   
                                             4.000000      4.0  0.690576   
                                             inf           4.0  0.692706   
                     True                    2.000000      4.0  0.690960   
                                             3.000000      4.0  0.690278   
                                             4.000000      4.0  0.681932   
                                             inf           4.0  0.670714   
1023       False     False                   2.000000      4.0  0.715963   
                                             3.000000      4.0  0.736660   
                                             4.000000      4.0  0.722767   
                                             inf           4.0  0.726727   
                     True                    2.000000      4.0  0.714170   
                                             3.000000      4.0  0.735094   
                                             4.000000      4.0  0.717954   
                                             inf           4.0  0.711995   
2047       False     False                   2.000000      4.0  0.763530   
                                             3.000000      4.0  0.743932   
                                             4.000000      4.0  0.734666   
                                             inf           4.0  0.731092   
                     True                    2.000000      4.0  0.764766   
                                             3.000000      4.0  0.742292   
                                             4.000000      4.0  0.729244   
                                             inf           4.0  0.710857   
4091       False     False                   2.000000      4.0  0.786279   
                                             3.000000      4.0  0.794000   
                                             4.000000      4.0  0.793129   
                                             inf           4.0  0.770587   
                     True                    2.000000      4.0  0.782616   
                                             3.000000      4.0  0.791856   
...                                                        ...       ...   
8191       False     False                   4.000000      4.0  0.813130   
                                             inf           4.0  0.799811   
                     True                    2.000000      4.0  0.787388   
                                             3.000000      4.0  0.808960   
                                             4.000000      4.0  0.810137   
                                             inf           4.0  0.777858   
16383      False     False                   2.000000      4.0  0.793173   
                                             3.000000      4.0  0.804440   
                                             4.000000      4.0  0.814413   
                                             inf           4.0  0.806804   
                     True                    2.000000      4.0  0.794044   
                                             3.000000      4.0  0.808781   
                                             4.000000      4.0  0.815069   
                                             inf           4.0  0.793290   
32767      False     False                   2.000000      4.0  0.793855   
                                             3.000000      4.0  0.812485   
                                             4.000000      4.0  0.820421   
                                             inf           4.0  0.823646   
                     True                    2.000000      4.0  0.793909   
                                             3.000000      4.0  0.811710   
                             

In [7]:
# What about scaling?
# What about removing duplicates ("representative columns")?
# What about hashing each substructure to different columns?
# What about L1 (note that, as usual, it works worse than L1)?
# What about the same analysis with our original SMARTS feats? (computation needs to be better,
#   - What about completing with other substructures (e.g. our not so clever add FCFP to the mix?).
# What about just using FCFP itself?
# What about RF, now that sklearn supports CSR for building them?
# What about using the malaria dataset I mined out of chembl, so we can trust a bit more the results?